# Deriving Player Batting Value

In an exercise in analysing MLB play-by-play data, I have derived from scratch a metric to evaluate player batting performance, finding the overall value (in terms of runs value added) that each player provided at the plate.

In [227]:
import pandas as pd
import numpy as np

In [228]:
april = pd.read_csv('events_apr.csv')
may = pd.read_csv('events_may.csv')
june = pd.read_csv('events_jun.csv')
july = pd.read_csv('events_jul.csv')
august = pd.read_csv('events_aug.csv')
september = pd.read_csv('events_sep.csv')

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (86) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [229]:
pbp = april
pbp = pbp.append(may).append(june).append(july).append(august).append(september)

pd.set_option('display.max_columns', 200)
pbp.head(5)

,GAME_ID,YEAR_ID,AWAY_TEAM_ID,INN_CT,BAT_HOME_ID,OUTS_CT,BALLS_CT,STRIKES_CT,PITCH_SEQ_TX,AWAY_SCORE_CT,HOME_SCORE_CT,BAT_ID,BAT_HAND_CD,RESP_BAT_ID,RESP_BAT_HAND_CD,PIT_ID,PIT_HAND_CD,RES_PIT_ID,RES_PIT_HAND_CD,POS2_FLD_ID,POS3_FLD_ID,POS4_FLD_ID,POS5_FLD_ID,POS6_FLD_ID,POS7_FLD_ID,POS8_FLD_ID,POS9_FLD_ID,BASE1_RUN_ID,BASE2_RUN_ID,BASE3_RUN_ID,EVENT_TX,LEADOFF_FL,PH_FL,BAT_FLD_CD,BAT_LINEUP_ID,EVENT_CD,BAT_EVENT_FL,AB_FL,H_CD,SH_FL,SF_FL,EVENT_OUTS_CT,DP_FL,TP_FL,RBI_CT,WP_FL,PB_FL,FLD_CD,BATTEDBALL_CD,BUNT_FL,FOUL_FL,BATTEDBALL_LOC_TX,ERR_CT,ERR1_FLD_CD,ERR1_CD,ERR2_FLD_CD,ERR2_CD,ERR3_FLD_CD,ERR3_CD,BAT_DEST_ID,RUN1_DEST_ID,RUN2_DEST_ID,RUN3_DEST_ID,BAT_PLAY_TX,RUN1_PLAY_TX,RUN2_PLAY_TX,RUN3_PLAY_TX,RUN1_SB_FL,RUN2_SB_FL,RUN3_SB_FL,RUN1_CS_FL,RUN2_CS_FL,RUN3_CS_FL,RUN1_PK_FL,RUN2_PK_FL,RUN3_PK_FL,RUN1_RESP_PIT_ID,RUN2_RESP_PIT_ID,RUN3_RESP_PIT_ID,GAME_NEW_FL,GAME_END_FL,PR_RUN1_FL,PR_RUN2_FL,PR_RUN3_FL,REMOVED_FOR_PR_RUN1_ID,REMOVED_FOR_PR_RUN2_ID,REMOVED_FOR_PR_RUN3_ID,REMOVED_FOR_PH_BAT_ID,REMOVED_FOR_PH_BAT_FLD_CD,PO1_FLD_CD,PO2_FLD_CD,PO3_FLD_CD,ASS1_FLD_CD,ASS2_FLD_CD,ASS3_FLD_CD,ASS4_FLD_CD,ASS5_FLD_CD,EVENT_ID,HOME_TEAM_ID,BAT_TEAM_ID,FLD_TEAM_ID,BAT_LAST_ID,INN_NEW_FL,INN_END_FL,START_BAT_SCORE_CT,START_FLD_SCORE_CT,INN_RUNS_CT,GAME_PA_CT,INN_PA_CT,PA_NEW_FL,PA_TRUNC_FL,START_BASES_CD,END_BASES_CD,BAT_START_FL,RESP_BAT_START_FL,BAT_ON_DECK_ID,BAT_IN_HOLD_ID,PIT_START_FL,RESP_PIT_START_FL,RUN1_FLD_CD,RUN1_LINEUP_ID,RUN1_ORIGIN_EVENT_ID,RUN2_FLD_CD,RUN2_LINEUP_ID,RUN2_ORIGIN_EVENT_ID,RUN3_FLD_CD,RUN3_LINEUP_ID,RUN3_ORIGIN_EVENT_ID,RUN1_RESP_CATCH_ID,RUN2_RESP_CATCH_ID,RUN3_RESP_CATCH_ID,PA_BALL_CT,PA_CALLED_BALL_CT,PA_INTENT_BALL_CT,PA_PITCHOUT_BALL_CT,PA_HIT_BALL_CT,PA_OTHER_BALL_CT,PA_STRIKE_CT,PA_CALLED_STRIKE_CT,PA_SWINGMISS_STRIKE_CT,PA_FOUL_STRIKE_CT,PA_BIP_STRIKE_CT,PA_OTHER_STRIKE_CT,EVENT_RUNS_CT,FLD_ID,BASE2_FORCE_FL,BASE3_FORCE_FL,BASE4_FORCE_FL,BAT_SAFE_ERR_FL,BAT_FATE_ID,RUN1_FATE_ID,RUN2_FATE_ID,RUN3_FATE_ID,FATE_RUNS_CT,ASS6_FLD_CD,ASS7_FLD_CD,ASS8_FLD_CD,ASS9_FLD_CD,ASS10_FLD_CD,UNKNOWN_OUT_EXC_FL,UNCERTAIN_PLAY_EXC_FL
0,ARI201704020,2017,SFN,1,0,0,1,1,CBX,0,0,spand001,L,spand001,L,greiz001,R,greiz001,R,mathj001,goldp001,drurb001,lambj001,owinc001,tomay001,polla001,perad001,NaN,NaN,NaN,NaN,T,F,8,1,2,T,T,0,F,F,1,F,F,0,F,F,0,P,F,F,NaN,0,0,N,0,N,0,N,0,0,0,0,NaN,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,T,F,F,F,F,NaN,NaN,NaN,NaN,0,6,0,0,0,0,0,0,0,1,ARI,SFN,ARI,0,T,F,0,0,0,0,0,T,F,0,0,T,T,beltb001,pench001,T,T,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,1,1,0,0,0,0,2,1,0,0,1,0,0,NaN,F,F,F,F,0,0,0,0,0,0,0,0,0,0,F,F
1,ARI201704020,2017,SFN,1,0,1,3,2,FFBBBB,0,0,beltb001,L,beltb001,L,greiz001,R,greiz001,R,mathj001,goldp001,drurb001,lambj001,owinc001,tomay001,polla001,perad001,NaN,NaN,NaN,NaN,F,F,3,2,14,T,F,0,F,F,0,F,F,0,F,F,0,NaN,F,F,NaN,0,0,N,0,N,0,N,1,0,0,0,NaN,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,F,F,F,F,F,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,2,ARI,SFN,ARI,0,F,F,0,0,0,1,1,T,F,0,1,T,T,pench001,poseb001,T,T,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,4,4,0,0,0,0,2,0,0,2,0,0,0,NaN,F,F,F,F,2,0,0,0,0,0,0,0,0,0,F,F
2,ARI201704020,2017,SFN,1,0,1,1,1,BSX,0,0,pench001,R,pench001,R,greiz001,R,greiz001,R,mathj001,goldp001,drurb001,lambj001,owinc001,tomay001,polla001,perad001,beltb001,NaN,NaN,NaN,F,F,9,3,20,T,T,1,F,F,0,F,F,0,F,F,8,G,F,F,NaN,0,0,N,0,N,0,N,1,2,0,0,NaN,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,greiz001,NaN,NaN,F,F,F,F,F,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,3,ARI,SFN,ARI,0,F,F,0,0,0,2,2,T,F,1,3,T,T,poseb001,crawb001,T,T,3,2,2,0,0,0,0,0,0,mathj001,NaN,NaN,1,1,0,0,0,0,2,0,1,0,1,0,0,polla001,F,F,F,F,1,2,0,0,0,0,0,0,0,0,F,F
3,ARI201704020,2017,SFN,1,0,1,1,2,*BFCC,0,0,poseb001,R,poseb001,R,greiz001,R,greiz001,R,mathj001,goldp001,drurb001,lambj001,owinc001,tomay001,polla001,perad001,pench001,beltb001,NaN,K,F,F,2,4,3,T,T,0,F,F,1,F,F,0,F,F,0,NaN,F,F,NaN,0,0,N,0,N,0,N,0,1,2,0,2.0,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,greiz001,greiz001,NaN,F,F,F,F,F,NaN,NaN,NaN,NaN,0,2,0,0,0,0,0,0,0,4,ARI,SFN,ARI,0,F,F,0,0,0,3,3,T,F,3,3,T,T,crawb001,nunee002,T,T,9,3,3,3,2,2,0,0,0,m

In [230]:
pbp = pbp.rename(columns=lambda x: x.strip())

In [231]:
pbp['HOME_TEAM_ID'].unique()

array(['ARI', 'SLN', 'TBA', 'BAL', 'BOS', 'CIN', 'HOU', 'LAN', 'MIL',
       'MIN', 'NYN', 'OAK', 'TEX', 'WAS', 'CHA', 'ANA', 'COL', 'DET',
       'PHI', 'PIT', 'SDN', 'CHN', 'KCA', 'NYA', 'SEA', 'SFN', 'CLE',
       'MIA', 'TOR', 'ATL'], dtype=object)

It will be useful at some points to separate the American and National leagues, which have meaningfully different run environments due to the DH.

In [232]:
AL = ['TBA','BAL','BOS','HOU','MIN','OAK','TEX','ANA','CHA','DET','KCA','NYA','SEA','CLE','TOR']
NL = ['ARI','SLN','CIN','LAN','MIL','NYN','WAS','COL','PHI','PIT','SDN','CHN','SFN','MIA','ATL']

# Run Values

When a player comes to the plate, any number of things can happen: strikeout, walk, single, double, etc. But how valuable are each of those events? It is obvious that a double is prefereable to a single, but exacly how much more valuable is it? Fans often reference the traditional "triple slash line" to gauge a player's batting skill: batting average, on base percentage, slugging percentage. Each of these have implicit values for each event.

AVG: Batting average values all hits the same, with walks and HBP being irrelevant, and all types of outs being equal.

OBP: On base percentage counts all times on base as equal, with a single being equal to a walk and a home run.

SLG: Similar to batting average but total bases are taken into account, so a double is worth twice as much as a single, and a home run four times as much.

The general idea is that AVG is a good measure of hitting ability, while SLG takes power into account, and OBP measures overall hitting ability with plate discipline taken into account. One useful stat is OPS to combine some of the strengths of OBP and SLG. 

But implicit in all of these stats are the assumptions above, which leads to some issues. At the two ends of the extreme, AVG and OBP value a single and a double the same, but SLG values a double twice as much. Which one is correct, or is it somehwere in between? There must be a better way to derive the relative values of each event.

That is where run values come in. The basis works with base-out-states. The three out states are 0 outs, 1 out, and 2 out. There are 8 base states: no one on, runner on first, runner on first and second, etc. Combining these means there are 24 total base out states. It is useful to determine how many runs can be expected from a given base-out-state. The best way to do this is to look at the play-by-play data and separate each event by their base-out-state and determine how many runs were scored from that point until the end of the inning. Doing this for each event can reveal the overall run-scoring picture.

In [233]:
pbp['HALF_INN_ID'] = pbp['GAME_ID'].map(str)+pbp['INN_CT'].map(str)+pbp['BAT_HOME_ID'].map(str)
end_of_inning = pbp[pbp['INN_END_FL'] == 'T']

half_innings = list(end_of_inning['HALF_INN_ID'])
inn_end_scores = list(end_of_inning['INN_RUNS_CT'])
inn_end_scores_dict = dict(zip(half_innings, inn_end_scores))

In [234]:
pbp['INN_END_SCORE'] = pbp['HALF_INN_ID'].map(inn_end_scores_dict)

In [235]:
pbp['RUNS_REST_OF_INN'] = pbp['INN_END_SCORE'] - pbp['INN_RUNS_CT'] + pbp['EVENT_RUNS_CT']

In [236]:
run_expect = pbp.groupby(['START_BASES_CD','OUTS_CT'])['RUNS_REST_OF_INN'].mean()

In [237]:
base_state_dict = {0:'Empty',1:'1B _ _',2:'_ 2B _',3:'1B 2B _',
                  4: '_ _ 3B',5:'1B _ 3B',6:'_ 2B 3B',7:'1B 2B 3B'}

In [238]:
pbp['BASE_STATE'] = pbp['START_BASES_CD'].map(base_state_dict)
pbp['END_BASE_STATE'] = pbp['END_BASES_CD'].map(base_state_dict)

In [239]:
event_code_dict = {0:'Unknown event',1:'No event',2:'Generic out',3:'Strikeout',4:'Stolen base',5:'Defensive indifference',
                  6:'Caught stealing',7:'Pickoff error',8:'Pickoff',9:'Wild pitch',10:'Passed ball',11:'Balk',12:'Other advance',
                  13:'Foul error',14:'Walk',15:'Intentional walk',16:'Hit by pitch',17:'Interference',18:'Error',
                   19:'Fielders choice',20:'Single',21:'Double',22:'Triple',23:'Home Run',24:'Missing play'}

In [240]:
pbp['EVENT_STR'] = pbp['EVENT_CD'].map(event_code_dict)

## Run Expectancy

Below is a table displaying on average how many runs were scored from the point that a certain base-out-state occurred until the end of the inning. For example, referencing the first value in the table, across the total occurrences of a plate appearance with bases loaded and 0 outs in 2017, an average of 2.23 runs were scored between the beginning of that occurrence through the end of the inning. The run expectancy values alight with intuition, with runs being more likely when there are fewer out or when runners on base are greater in number or closer to home plate.

A good way to validate these values is to refer to the overall run environment in 2017:

https://www.baseball-reference.com/leagues/MLB/2017.shtml

Overall, the average per team was runs/game was 4.647, or roughly 0.516 runs/inning. This compares closely to the 0.521 given in the table for bases empty and 0 outs, which it should because that is how every inning begins.

In [241]:
pbp.groupby(['BASE_STATE','OUTS_CT']).mean()['RUNS_REST_OF_INN']

BASE_STATE  OUTS_CT
1B 2B 3B    0          2.232955
            1          1.635198
            2          0.804745
1B 2B _     0          1.510513
            1          0.954603
            2          0.453271
1B _ 3B     0          1.833333
            1          1.220179
            2          0.504332
1B _ _      0          0.919089
            1          0.531170
            2          0.229446
Empty       0          0.521534
            1          0.274955
            2          0.109338
_ 2B 3B     0          2.052326
            1          1.493859
            2          0.584436
_ 2B _      0          1.175439
            1          0.721853
            2          0.330587
_ _ 3B      0          1.409382
            1          0.981100
            2          0.382867
Name: RUNS_REST_OF_INN, dtype: float64

In [242]:
run_expect[7][2]

0.80474542758279788

In [243]:
run_expect = pbp.groupby(['START_BASES_CD','OUTS_CT'])['RUNS_REST_OF_INN'].mean().unstack()
run_expect[3] = 0
run_expect = run_expect.stack()#.to_frame('Run Expectancy')

In [244]:
pbp['END_OUTS_CT'] = pbp['OUTS_CT'] + pbp['EVENT_OUTS_CT']

In [245]:
def run_value_start(start_base_state,start_outs):
    return run_expect[start_base_state][start_outs]

def run_value_end(end_base_state,end_outs):
    return run_expect[end_base_state][end_outs]

def run_value(run_value_start,run_value_end,runs):
    return run_value_end - run_value_start + runs

pbp['START_RUN_VALUE'] = pbp.apply(lambda x:run_value_start(x['START_BASES_CD'],x['OUTS_CT']), axis=1)
pbp['END_RUN_VALUE'] = pbp.apply(lambda x:run_value_end(x['END_BASES_CD'],x['END_OUTS_CT']), axis=1)
pbp['RUN_VALUE'] = pbp.apply(lambda x: run_value(x['START_RUN_VALUE'],x['END_RUN_VALUE'],x['EVENT_RUNS_CT']), axis=1)

## Run Values

With the run expectancy values derived above, we can now look deeper into the value of each event. Each event moves the state of the inning from one base-out-state to another. This means that the run expectancy also is moved from one value to another. We can take the difference of these run expectancies to attain a run value for each event, or the amount gained or lost in run expectancy. The run value of an event is also credited with any runs that it directly drove in.

In [246]:
start = pbp.iloc[0]['START_BASES_CD']
end = pbp.iloc[0]['END_BASES_CD']
outs = pbp.iloc[0]['OUTS_CT']
end_outs = pbp.iloc[0]['END_OUTS_CT']
print(run_expect[end][end_outs]-run_expect[start][outs])

-0.246578194734


## Play Parser

In [247]:
players = pd.read_csv('Player_IDs.csv',encoding = "ISO-8859-1")
players.head()

,ID,Last,First,Player debut,Mgr debut,Coach debut,Ump debut
0,aardd001,Aardsma,David,4/6/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,4/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,4/10/1962,NaN,4/6/1979,NaN
3,aased001,Aase,Don,7/26/1977,NaN,NaN,NaN
4,abada001,Abad,Andy,9/10/2001,NaN,NaN,NaN


In [248]:
player_dict = dict(zip(players['ID'],(players['First']+' '+players['Last']))) #dictionary between IDs and first and last name
pbp['BATTER'] = pbp['BAT_ID'].map(player_dict)
pbp['PITCHER'] = pbp['PIT_ID'].map(player_dict)

In [249]:
def play_parser(play):
    if play['BAT_HOME_ID'] == 0:
        inning = 'Top'
    else:
        inning = 'Bottom'
    print(inning,play['INN_CT'],',',play['HOME_TEAM_ID'],play['HOME_SCORE_CT'],',',play['AWAY_TEAM_ID'],play['AWAY_SCORE_CT'],',',play['OUTS_CT'],' out',',',play['BASE_STATE'])
    print('Batter:',play['BATTER'])
    print('Pitcher:',play['PITCHER'])
    print('Play Result:',play['EVENT_STR'],',',play['EVENT_OUTS_CT'],' out(s)',',',play['EVENT_RUNS_CT'],' run(s)',',',play['END_BASE_STATE'])
    print('Run value:',play['RUN_VALUE'])
    print('Expected inning runs:',play['INN_RUNS_CT']+play['END_RUN_VALUE'])

In [250]:
play_parser(pbp.iloc[682])

Bottom 5 , MIL 4 , COL 4 , 0  out , _ 2B _
Batter: Ryan Braun
Pitcher: Scott Oberg
Play Result: Strikeout , 1  out(s) , 0  run(s) , _ 2B _
Run value: -0.453585514173
Expected inning runs: 4.72185308232


## Run Value Leaders

Looking at the top 10 (and bottom 10) players in run values produced, appropriate names pop up. Great hitters like Joey Votto, Charlie Blackmon, and Nolan Arenado are at the top, and players who struggled like Rougned Odor are at the bottom. However, a few things here illustrate that some more adjustments may be helpful. 

1. The biggest thing that jumps out is the two Rockies players in the 2nd and 3rd positions. While Blackmon and Arenado are great hitters who had big years, Coors Field is famous for having an elevated run environment. We will explore that a bit further. 
2. Another thing to note is that some of the possible events should not actually be credited to the batter. For example, Joey Votto may be getting credit for some Billy Hamilton steals here, since he happened to be at the plate at the time.
3. Awarding players the exact run value (change in run expectancy for the inning) might not be the best methodology. This value will be different for certain events based on initial base-out state, which the batter has no control over. It would be better to group run values by batting event for a consistent value for each event.

In [251]:
pbp.groupby('BATTER')['RUN_VALUE'].sum().sort_values(ascending=False).head(10)

BATTER
Joey Votto           66.399203
Charlie Blackmon     63.633584
Nolan Arenado        59.340572
Aaron Judge          56.032800
Paul Goldschmidt     54.863303
Giancarlo Stanton    51.295499
Mike Trout           50.275027
Nelson Cruz          48.411021
Marcell Ozuna        46.152749
Bryce Harper         43.465397
Name: RUN_VALUE, dtype: float64

In [252]:
pbp.groupby('BATTER')['RUN_VALUE'].sum().sort_values().head(10)

BATTER
Rougned Odor       -34.370503
Tyler Saladino     -29.815104
Trevor Plouffe     -29.429912
Billy Hamilton     -29.171518
Tim Anderson       -29.057075
Martin Maldonado   -27.749848
Kevin Pillar       -23.896748
Darwin Barney      -22.072913
Alex Gordon        -21.813842
Adam Engel         -21.704612
Name: RUN_VALUE, dtype: float64

## Event Values

Once the run value for every row in the play by play is calculated, they can be grouped by events. The average is the average run value for each type of event in 2017. So for instance, a single, on average, added 0.45 runs to a team,s expected run value for the inning based on how it changed the base-out-state.

This proves to be very useful information. For example, we can see that based on these event values, a double is not 2 times as valuable as a single but 1.71 times as valuable. Based on the value of a stolen base relative to the negative value of a caught stealing, a runner should be able to steal a base at at least a 70% success rate (probably more depending on the situation) for it to really be worth it.

In [253]:
event_value = pbp.groupby('EVENT_STR')['RUN_VALUE'].mean().sort_values(ascending=False)
print(event_value)

EVENT_STR
Home Run                  1.377935
Triple                    1.058986
Double                    0.777538
Error                     0.497508
Single                    0.454261
Interference              0.398590
Hit by pitch              0.335934
Walk                      0.315645
Balk                      0.286674
Passed ball               0.273919
Wild pitch                0.271290
Stolen base               0.194913
Intentional walk          0.173017
Defensive indifference    0.111902
Foul error                0.000000
Pickoff                  -0.216529
Strikeout                -0.277981
Generic out              -0.282517
Fielders choice          -0.316503
Caught stealing          -0.426112
Other advance            -0.528403
Name: RUN_VALUE, dtype: float64


In [254]:
def event_run_value(event):
    return event_value[event]

pbp['EVENT_RUN_VALUE'] = pbp.apply(lambda x: event_run_value(x['EVENT_STR']), axis=1)

In [255]:
pbp.groupby('BATTER')[['EVENT_RUN_VALUE','RUN_VALUE']].sum().sort_values(by='EVENT_RUN_VALUE',ascending=False).head(10)

,EVENT_RUN_VALUE,RUN_VALUE
BATTER,,
Aaron Judge,66.311251,56.032800
Joey Votto,65.618470,66.399203
Charlie Blackmon,59.171884,63.633584
Giancarlo Stanton,55.835069,51.295499
Mike Trout,52.460135,50.275027
Paul Goldschmidt,47.945671,54.863303
Jose Altuve,46.462056,37.767092
Kris Bryant,45.823214,35.599932
J.D. Martinez,45.645107,35.794164


In [256]:
(pbp[pbp['BATTER'] == 'Aaron Judge'].groupby('EVENT_STR')['RUN_VALUE'].mean().sort_values(ascending=False)/event_value).sort_values(ascending=False)

EVENT_STR
Fielders choice           2.162881
Pickoff                   1.948156
Caught stealing           1.250806
Wild pitch                1.154035
Generic out               1.114816
Triple                    1.079768
Home Run                  1.032322
Intentional walk          1.018321
Double                    1.016435
Strikeout                 1.010258
Hit by pitch              0.962152
Single                    0.948876
Walk                      0.925905
Stolen base               0.915676
Defensive indifference    0.903834
Error                     0.770098
Passed ball               0.478847
Balk                           NaN
Foul error                     NaN
Interference                   NaN
Other advance                  NaN
Name: RUN_VALUE, dtype: float64

## Non-batting Events

In [257]:
pbp[pbp['BATTER'] == 'Joey Votto']['EVENT_STR'].value_counts()

Generic out               299
Walk                      114
Single                    108
Strikeout                  83
Home Run                   36
Double                     34
Intentional walk           20
Stolen base                19
Hit by pitch                8
Caught stealing             3
Wild pitch                  3
Error                       3
Pickoff                     1
Foul error                  1
Defensive indifference      1
Passed ball                 1
Fielders choice             1
Triple                      1
Balk                        1
Name: EVENT_STR, dtype: int64

It does appear that some things are misaccredited to the batter. Joey Votto has 19 steals to his name here, when in reality he only had 5 last season. He is not being credited with any of his own 5 and has 19 'stolen' steals. Crediting stolen bases to the correct player is a bit tricky but for the purposes of value at the plate they are not relevant anyway, so this will be adjusted.

In [258]:
pbp['EVENT_STR'].value_counts()

Generic out               83288
Strikeout                 40104
Single                    26918
Walk                      14858
Double                     8392
Home Run                   6105
Stolen base                2210
Hit by pitch               1764
Wild pitch                 1655
Error                      1630
Intentional walk            970
Triple                      796
Caught stealing             691
Pickoff                     444
Fielders choice             427
Passed ball                 336
Defensive indifference      285
Balk                        155
Other advance                86
Interference                 43
Foul error                   35
Name: EVENT_STR, dtype: int64

In [259]:
batting_events = ['Generic out','Strikeout','Single','Walk','Double','Home Run','Hit by pitch',
                 'Wild pitch','Intentional walk','Triple','Fielders choice','Passed ball',
                 'Defensive indifference','Balk']

In [260]:
batting = pbp[pbp['EVENT_STR'].isin(batting_events)]

In [261]:
batting.groupby('BATTER')['EVENT_RUN_VALUE'].sum().sort_values(ascending=False).head(10)

BATTER
Aaron Judge          62.271367
Joey Votto           61.917467
Charlie Blackmon     55.444636
Giancarlo Stanton    50.547775
Mike Trout           49.065212
Jose Altuve          44.761126
J.D. Martinez        43.907890
Paul Goldschmidt     43.868858
Kris Bryant          43.183994
Nolan Arenado        40.266773
Name: EVENT_RUN_VALUE, dtype: float64

## Park Factors

One of the fun things about baseball is that every ballpark is different. One of the byproducts of this is that some parks lend themselves to higher scores than others. This makes it a bit trickier to compare players accross teams, however. Because some parks have an overall higher run environment, it is misleading to judge each player based on the league average run values. It may be better to determine run values for each park instead to better normalize for park.

One way to approach park factors is to find how many runs each team scored at home vs. on the road (an example for CLE is below). Since its the same team batting in each instance, it should be a reasonable way to how much runs scored can be attributed to the park. By dividing these home/away splits, we should be able to determine how much to adjust the value of an event based on the park that it occurred in. If it happened in a good hitters park, dock it a bit, etc.

In [262]:
home_away_splits = pbp.groupby(['BAT_TEAM_ID','BAT_HOME_ID']).sum()['EVENT_RUNS_CT']
home_away_splits['CLE']

BAT_HOME_ID
0    412
1    406
Name: EVENT_RUNS_CT, dtype: int64

In [263]:
park_factor = {}
for team in pbp['HOME_TEAM_ID'].unique():
    park_factor[team] = home_away_splits[team][1]/home_away_splits[team][0]
park_factor

{'ANA': 1.0056497175141244,
 'ARI': 1.2873239436619719,
 'ATL': 0.89637305699481862,
 'BAL': 1.1350574712643677,
 'BOS': 0.97236180904522618,
 'CHA': 1.0764705882352941,
 'CHN': 1.1295336787564767,
 'CIN': 1.1452991452991452,
 'CLE': 0.9854368932038835,
 'COL': 1.4523809523809523,
 'DET': 1.3040752351097178,
 'HOU': 0.78842315369261473,
 'KCA': 0.93388429752066116,
 'LAN': 1.0698924731182795,
 'MIA': 0.91625615763546797,
 'MIL': 1.0165289256198347,
 'MIN': 1.0632911392405062,
 'NYA': 1.1081081081081081,
 'NYN': 0.87979539641943738,
 'OAK': 1.2950310559006211,
 'PHI': 1.15625,
 'PIT': 0.97050147492625372,
 'SDN': 1.0,
 'SEA': 0.97368421052631582,
 'SFN': 0.97832817337461297,
 'SLN': 0.89303482587064675,
 'TBA': 0.98285714285714287,
 'TEX': 1.2828571428571429,
 'TOR': 0.91966759002770082,
 'WAS': 1.0577889447236182}

In [264]:
pbp.groupby('BAT_HOME_ID')['EVENT_RUNS_CT'].sum()

BAT_HOME_ID
0    11030
1    11552
Name: EVENT_RUNS_CT, dtype: int64

However, there are several issues with this method.

1. Comparing a teams runs produced at home vs. on the road is lacking because to assume that a team's performance on the road is a good baseline for their "true" output is probably not wise.
2. It treats all events the same. It could be that home runs more valuable than average at one park, but triples are less valuable. This method would either reward or punish both.

So instead, I will develop a method that only looks at what happens in each park (as opposed to trying to work with home/away splits) and will develop factors not just for each park, but for each event in each park.

Below is an example of the run average run values of events at Coors Field (to pick an extreme).

In [265]:
park_event_value = pbp.groupby(['HOME_TEAM_ID','EVENT_STR'])['RUN_VALUE'].mean()
print(park_event_value['COL'].sort_values(ascending=False))

EVENT_STR
Home Run                  1.401738
Triple                    1.041954
Double                    0.826277
Error                     0.528586
Single                    0.487027
Interference              0.423432
Walk                      0.324483
Hit by pitch              0.319264
Passed ball               0.256511
Balk                      0.232634
Wild pitch                0.231019
Intentional walk          0.187427
Stolen base               0.179719
Defensive indifference    0.144713
Pickoff                  -0.281423
Strikeout                -0.282918
Generic out              -0.290622
Fielders choice          -0.399152
Caught stealing          -0.431073
Other advance            -0.685830
Name: RUN_VALUE, dtype: float64


To get a better estimate at how valuable each event is in each park compared to the average, I will also factor in the overall run environment for that park. A certain run value is less valuable in a park if more runs are scored there per inning. So runs/inning for each park will be factored into the method.

In [266]:
park_runs = pbp.groupby(['HOME_TEAM_ID']).sum()['EVENT_RUNS_CT']
park_inn = pbp.groupby('HOME_TEAM_ID')['INN_NEW_FL'].value_counts()
park_rpi = {}
for team in pbp['HOME_TEAM_ID'].unique():
    park_rpi[team] = park_runs[team]/park_inn[team]['T']

In [267]:
park_rpi

{'ANA': 0.47655172413793101,
 'ARI': 0.55958188153310107,
 'ATL': 0.528236914600551,
 'BAL': 0.54336043360433606,
 'BOS': 0.4946236559139785,
 'CHA': 0.53314724354501042,
 'CHN': 0.55786555786555792,
 'CIN': 0.56678200692041525,
 'CLE': 0.47622377622377621,
 'COL': 0.63726182074805926,
 'DET': 0.60593512767425806,
 'HOU': 0.50313588850174218,
 'KCA': 0.50244926522043387,
 'LAN': 0.46341463414634149,
 'MIA': 0.51138716356107661,
 'MIL': 0.50827586206896547,
 'MIN': 0.58113730929264906,
 'NYA': 0.53566433566433569,
 'NYN': 0.51143451143451146,
 'OAK': 0.56503785271851337,
 'PHI': 0.52084757347915245,
 'PIT': 0.46580027359781123,
 'SDN': 0.45028011204481794,
 'SEA': 0.50068775790921594,
 'SFN': 0.43771043771043772,
 'SLN': 0.47586206896551725,
 'TBA': 0.45787292817679559,
 'TEX': 0.61656228253305501,
 'TOR': 0.50174703004891685,
 'WAS': 0.53342618384401119}

In [268]:
total_rpi = pbp['EVENT_RUNS_CT'].sum()/(pbp['INN_NEW_FL'].value_counts()['T'])
print(total_rpi)

0.520586472405


In [269]:
event_value_adj = event_value/total_rpi

In [270]:
park_event_value_adj = {}
for park in pbp['HOME_TEAM_ID'].unique():
    park_event_value_adj[park] = park_event_value[park]/park_rpi[park]

In [271]:
park_event_factor = park_event_value_adj
for park in pbp['HOME_TEAM_ID'].unique():
    park_event_factor[park] = park_event_value_adj[park].sort_values(ascending=False)/event_value_adj

Below is an example of the final park event factors for Colorado. Most of the values are below 1, meaning that most events are less valuable in Colorado than they are on average. These values will be multiplied by the existing run values for each event, depending on park, to get the park adjusted value. So using this example, a home run in Colorado will have its run value multiplied by 0.83, giving it a bit less value, since it had less effect compared to the total run environment in the park.

In [272]:
park_event_factor['COL']

EVENT_STR
Balk                      0.662916
Caught stealing           0.826423
Defensive indifference    1.056432
Double                    0.868119
Error                     0.867942
Fielders choice           1.030232
Foul error                     NaN
Generic out               0.840348
Hit by pitch              0.776373
Home Run                  0.831023
Intentional walk          0.884948
Interference              0.867826
Other advance             1.060295
Passed ball               0.764998
Pickoff                   1.061740
Single                    0.875836
Stolen base               0.753231
Strikeout                 0.831421
Triple                    0.803772
Walk                      0.839783
Wild pitch                0.695645
Name: RUN_VALUE, dtype: float64

In [273]:
def park_adjust(event_rv,park,event):
    return event_rv*park_event_factor[park][event]

pbp['EVENT_PARK_RV'] = pbp.apply(lambda x: park_adjust(x['EVENT_RUN_VALUE'],x['HOME_TEAM_ID'],x['EVENT_STR']), axis=1)

After park adjustments, the leaderboard is shifted a bit more. Most players don't see too much change as most parks are still comparable, but players whose home parks are at an extreme are indeed observed to move the most. Charlie Blackmon lost about 8 runs, with Giancarlo Stanton gaining 5.

In [276]:
batting = pbp[pbp['EVENT_STR'].isin(batting_events)]
batting.groupby('BATTER')[['EVENT_PARK_RV','EVENT_RUN_VALUE']].sum().sort_values(by='EVENT_PARK_RV',ascending=False).head(10)

,EVENT_PARK_RV,EVENT_RUN_VALUE
BATTER,,
Aaron Judge,62.273449,62.271367
Joey Votto,60.001439,61.917467
Giancarlo Stanton,55.313878,50.547775
Mike Trout,51.440382,49.065212
Charlie Blackmon,47.021472,55.444636
Kris Bryant,42.807732,43.183994
Jose Altuve,42.532265,44.761126
J.D. Martinez,40.741791,43.907890
Paul Goldschmidt,39.321993,43.868858
